# ETL I Pair 23/08/23

1. Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. NOTA: nos piden que el código debe estar en una función. Al final deberemos tener un dataframe como el siguiente:

2. Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

- Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs.